In [ ]:
bd_name = "bd4"

!apt-get update -y
!apt-get install postgresql postgresql-contrib -y
!service postgresql start
!sudo -u postgres dropdb --if-exists "$bd_name"
!sudo -u postgres psql -c "ALTER USER postgres PASSWORD 'postgres';"
!sudo -u postgres createdb "$bd_name"

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://cli.github.com/packages stable InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [ ]:
script_sql = """

-- -----------------------------------------------------
-- Tabela Clã
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Cla (
    id_cla VARCHAR(5) PRIMARY KEY,
    nome VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela Vila
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Vila (
    id_vila VARCHAR(5) PRIMARY KEY,
    nome VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela Missao
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Missao (
    id_missao VARCHAR(5) PRIMARY KEY,
    descricao TEXT,
    ranking CHAR(1)
);

-- -----------------------------------------------------
-- Tabela Time
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Time (
    id_time VARCHAR(5) PRIMARY KEY,
    nome VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela Invocacao
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Invocacao (
    id_invocacao VARCHAR(5) PRIMARY KEY,
    nome_invocacao VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela Chakra
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Chakra (
    id_chakra VARCHAR(5) PRIMARY KEY,
    natureza VARCHAR(100),
    cor VARCHAR(50),
    rotacao VARCHAR(50)
);

-- -----------------------------------------------------
-- Tabela Jutsu
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Jutsu (
    id_jutsu VARCHAR(5) PRIMARY KEY,
    nome_jutsu VARCHAR(100),
    tipo VARCHAR(50)
);

-- -----------------------------------------------------
-- Tabela Arma
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Arma (
    id_arma VARCHAR(5) PRIMARY KEY,
    nome_arma VARCHAR(100),
    tipo VARCHAR(50)
);

-- -----------------------------------------------------
-- Tabela Selo
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Selo (
    id_selo VARCHAR(5) PRIMARY KEY,
    nome VARCHAR(100)
);

-- -----------------------------------------------------
-- Tabela KekkeiGenkai (agora pode referenciar Cla)
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS KekkeiGenkai (
    id_kekkei VARCHAR(5) PRIMARY KEY,
    nome_kekkei VARCHAR(100),
    id_cla VARCHAR(5),
    FOREIGN KEY (id_cla) REFERENCES Cla(id_cla)
);

-- -----------------------------------------------------
-- Tabela Sensei (agora pode referenciar Time)
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Sensei (
    id_sensei VARCHAR(5) PRIMARY KEY,
    id_time VARCHAR(5),
    FOREIGN KEY (id_time) REFERENCES Time(id_time)
);

-- -----------------------------------------------------
-- Tabela Ninja (agora pode referenciar Vila e Cla)
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Ninja (
    id_ninja VARCHAR(5) PRIMARY KEY,
    nome VARCHAR(100),
    idade INT,
    ranking CHAR(1),
    tipo CHAR(1),
    id_vila VARCHAR(5),
    id_cla VARCHAR(5),
    FOREIGN KEY (id_vila) REFERENCES Vila(id_vila),
    FOREIGN KEY (id_cla) REFERENCES Cla(id_cla)
);

-- -----------------------------------------------------
-- Demais tabelas com dependências
-- -----------------------------------------------------
CREATE TABLE IF NOT EXISTS Bijuu (
    id_bijuu VARCHAR(5) PRIMARY KEY,
    id_ninja VARCHAR(5),
    nome_bijuu VARCHAR(100),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja)
);

CREATE TABLE IF NOT EXISTS Hokage (
    id_ninja VARCHAR(5) PRIMARY KEY,
    id_vila VARCHAR(5),
    FOREIGN KEY (id_vila) REFERENCES Vila(id_vila),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja)
);

CREATE TABLE IF NOT EXISTS Mestres (
    id_aprendiz VARCHAR(5),
    id_mestre VARCHAR(5) PRIMARY KEY,
    especialidade VARCHAR(100),
    FOREIGN KEY (id_aprendiz) REFERENCES Ninja(id_ninja)
);

-- Tabelas de relacionamento
CREATE TABLE IF NOT EXISTS Ninja_Arma (
    id_ninja VARCHAR(5),
    id_arma VARCHAR(5),
    PRIMARY KEY (id_ninja, id_arma),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_arma) REFERENCES Arma(id_arma)
);

CREATE TABLE IF NOT EXISTS Ninja_Invocacao (
    id_ninja VARCHAR(5),
    id_invocacao VARCHAR(5),
    PRIMARY KEY (id_ninja, id_invocacao),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_invocacao) REFERENCES Invocacao(id_invocacao)
);

CREATE TABLE IF NOT EXISTS Ninja_Chakra (
    id_ninja VARCHAR(5),
    id_chakra VARCHAR(5),
    PRIMARY KEY (id_ninja, id_chakra),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_chakra) REFERENCES Chakra(id_chakra)
);

CREATE TABLE IF NOT EXISTS Jutsu_Chakra (
    id_jutsu VARCHAR(5),
    id_chakra VARCHAR(5),
    PRIMARY KEY (id_jutsu, id_chakra),
    FOREIGN KEY (id_jutsu) REFERENCES Jutsu(id_jutsu),
    FOREIGN KEY (id_chakra) REFERENCES Chakra(id_chakra)
);

CREATE TABLE IF NOT EXISTS Selo_Jutsu (
    id_selo VARCHAR(5),
    id_jutsu VARCHAR(5),
    PRIMARY KEY (id_selo, id_jutsu),
    FOREIGN KEY (id_selo) REFERENCES Selo(id_selo),
    FOREIGN KEY (id_jutsu) REFERENCES Jutsu(id_jutsu)
);

CREATE TABLE IF NOT EXISTS Ninja_Time (
    id_ninja VARCHAR(5),
    id_time VARCHAR(5),
    PRIMARY KEY (id_ninja, id_time),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_time) REFERENCES Time(id_time)
);

CREATE TABLE IF NOT EXISTS Time_Missao (
    id_missao VARCHAR(5),
    id_time VARCHAR(5),
    PRIMARY KEY (id_missao, id_time),
    FOREIGN KEY (id_missao) REFERENCES Missao(id_missao),
    FOREIGN KEY (id_time) REFERENCES Time(id_time)
);

CREATE TABLE IF NOT EXISTS Ninja_Kekkei (
    id_ninja VARCHAR(5),
    id_kekkei VARCHAR(5),
    PRIMARY KEY (id_ninja, id_kekkei),
    FOREIGN KEY (id_ninja) REFERENCES Ninja(id_ninja),
    FOREIGN KEY (id_kekkei) REFERENCES KekkeiGenkai(id_kekkei)
);

-- Inserção de dados

INSERT INTO Cla (id_cla, nome) VALUES
('C1', 'Uchiha'),
('C2', 'Hyuga'),
('C3', 'Senju'),
('C4', 'Nara'),
('C5', 'Akimichi'),
('C6', 'Yamanaka'),
('C7', 'Inuzuka'),
('C8', 'Aburame'),
('C9', 'Sarutobi'),
('C10', 'Senju')
ON CONFLICT (id_cla) DO NOTHING;

INSERT INTO Time (id_time, nome) VALUES
('T001', 'Time Kakashi'),
('T002', 'Time Asuma'),
('T003', 'Time Kurenai'),
('T004', 'Time Gai'),
('T005', 'Time Minato'),
('T006', 'Time Gaara'),
('T007', 'Time Temari'),
('T008', 'Time Kankuro'),
('T009', 'Time Baki'),
('T010', 'Time da Areia Elite'),
('T011', 'Time Mei Terumi'),
('T012', 'Time Chojuro'),
('T013', 'Time Zabuza'),
('T014', 'Time Espadachins da Névoa'),
('T015', 'Time da Névoa Elite')
ON CONFLICT (id_time) DO NOTHING;

INSERT INTO Vila(id_vila, nome) VALUES
('001', 'Vila da Folha'),
('002', 'Vila da Areia'),
('003', 'Vila da Névoa')
ON CONFLICT (id_vila) DO NOTHING;

INSERT INTO Missao (id_missao, descricao, ranking) VALUES
('M001','Proteger o Daimyo em viagem oficial','S'),
('M002','Neutralizar organização criminosa internacional','S'),
('M003','Capturar ninja renegado de alto nível','S'),
('M004','Resgatar reféns em território inimigo','S'),
('M005','Eliminar ameaça classe S nas fronteiras','S')
ON CONFLICT (id_missao) DO NOTHING;

INSERT INTO Ninja (id_ninja, nome, idade, ranking, tipo, id_vila, id_cla) VALUES
    -- Vila da Folha (001) - Clãs clássicos da Folha
    ('N001', 'Itachi Uchiha', 21, 'S', 'J', '001', 'C1'),
    ('N002', 'Sasuke Uchiha', 16, 'A', 'C', '001', 'C1'),
    ('N003', 'Shisui Uchiha', 19, 'S', 'J', '001', 'C1'),
    ('N004', 'Fugaku Uchiha', 40, 'A', 'J', '001', 'C1'),

    ('N005', 'Hinata Hyuga', 16, 'A', 'G', '001', 'C2'),
    ('N006', 'Neji Hyuga', 17, 'A', 'J', '001', 'C2'),
    ('N007', 'Hiashi Hyuga', 42, 'S', 'J', '001', 'C2'),

    ('N008', 'Hashirama Senju', 35, 'S', 'J', '001', 'C3'),
    ('N009', 'Tobirama Senju', 34, 'S', 'J', '001', 'C3'),
    ('N010', 'Tsunade Senju', 36, 'S', 'J', '001', 'C3'),

    ('N011', 'Shikamaru Nara', 16, 'A', 'C', '001', 'C4'),
    ('N012', 'Shikaku Nara', 32, 'S', 'J', '001', 'C4'),

    ('N013', 'Choji Akimichi', 16, 'C', 'G', '001', 'C5'),
    ('N014', 'Choza Akimichi', 34, 'C', 'J', '001', 'C5'),

    ('N015', 'Ino Yamanaka', 16, 'B', 'G', '001', 'C6'),
    ('N016', 'Inoichi Yamanaka', 35, 'A', 'J', '001', 'C6'),

    ('N017', 'Kiba Inuzuka', 16, 'B', 'G', '001', 'C7'),
    ('N018', 'Tsume Inuzuka', 33, 'A', 'J', '001', 'C7'),

    ('N019', 'Shino Aburame', 16, 'A', 'C', '001', 'C8'),
    ('N020', 'Shibi Aburame', 35, 'A', 'J', '001', 'C8'),

    ('N021', 'Hiruzen Sarutobi', 68, 'S', 'J', '001', 'C9'),
    ('N022', 'Asuma Sarutobi', 31, 'A', 'J', '001', 'C9'),
    ('N023', 'Konohamaru Sarutobi', 12, 'C', 'G', '001', 'C9'),

    ('N024', 'Nawaki Senju', 12, 'B', 'G', '001', 'C10'),
    ('N025', 'Arata Senju', 20, 'B', 'C', '001', 'C10'),

    -- Vila da Areia (002) - usando os mesmos 10 clãs já cadastrados
    ('N026', 'Gaara', 16, 'S', 'J', '002', 'C3'),
    ('N027', 'Temari', 17, 'A', 'J', '002', 'C4'),
    ('N028', 'Kankuro', 17, 'A', 'J', '002', 'C5'),
    ('N029', 'Baki', 35, 'A', 'J', '002', 'C4'),
    ('N030', 'Yashamaru', 30, 'B', 'C', '002', 'C6'),
    ('N031', 'Maki', 18, 'B', 'G', '002', 'C6'),
    ('N032', 'Shira', 20, 'A', 'C', '002', 'C5'),
    ('N033', 'Suna Shinobi A', 19, 'C', 'G', '002', 'C7'),
    ('N034', 'Suna Shinobi B', 21, 'C', 'C', '002', 'C8'),
    ('N035', 'Suna Médico', 23, 'B', 'G', '002', 'C8'),

    ('N036', 'Mei Terumi', 30, 'S', 'J', '003', 'C2'),
    ('N037', 'Zabuza Momochi', 26, 'A', 'J', '003', 'C1'),
    ('N038', 'Haku', 15, 'B', 'G', '003', 'C1'),
    ('N039', 'Chojuro', 20, 'A', 'C', '003', 'C4'),
    ('N040', 'Ao', 28, 'A', 'J', '003', 'C6'),
    ('N041', 'Suigetsu Hozuki', 18, 'B', 'C', '003', 'C3'),
    ('N042', 'Mangetsu Hozuki', 20, 'A', 'J', '003', 'C3'),
    ('N043', 'Kisame Hoshigaki', 32, 'S', 'J', '003', 'C7')
ON CONFLICT (id_ninja) DO NOTHING;
"""
with open("script.sql", "w") as f:
    f.write(script_sql)

!sudo -u postgres psql -d "$bd_name" -f script.sql

psql:script.sql:9: NOTICE:  relation "cla" already exists, skipping
CREATE TABLE
psql:script.sql:17: NOTICE:  relation "vila" already exists, skipping
CREATE TABLE
psql:script.sql:26: NOTICE:  relation "missao" already exists, skipping
CREATE TABLE
psql:script.sql:34: NOTICE:  relation "time" already exists, skipping
CREATE TABLE
psql:script.sql:42: NOTICE:  relation "invocacao" already exists, skipping
CREATE TABLE
psql:script.sql:52: NOTICE:  relation "chakra" already exists, skipping
CREATE TABLE
psql:script.sql:61: NOTICE:  relation "jutsu" already exists, skipping
CREATE TABLE
psql:script.sql:70: NOTICE:  relation "arma" already exists, skipping
CREATE TABLE
psql:script.sql:78: NOTICE:  relation "selo" already exists, skipping
CREATE TABLE
psql:script.sql:88: NOTICE:  relation "kekkeigenkai" already exists, skipping
CREATE TABLE
psql:script.sql:97: NOTICE:  relation "sensei" already exists, skipping
CREATE TABLE
psql:script.sql:112: NOTICE:  relation "ninja" already exists, skippi

In [ ]:
!pip install psycopg2-binary

import psycopg2
import pandas as pd

conn = psycopg2.connect(
    dbname=bd_name,
    user="postgres",
    password="postgres",
    host="localhost"
)

cur = conn.cursor()
query = "SELECT * FROM Cla;"
cur.execute(query)
df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
print(df)
# df.to_csv("Cla.csv", index=False)
# print("CSV gerado com sucesso!")
cur.close()
conn.close()

  id_cla      nome
0     C1    Uchiha
1     C2     Hyuga
2     C3     Senju
3     C4      Nara
4     C5  Akimichi
5     C6  Yamanaka
6     C7   Inuzuka
7     C8   Aburame
8     C9  Sarutobi
9    C10     Senju


In [ ]:
conn = psycopg2.connect(
    dbname=bd_name,
    user="postgres",
    password="postgres",
    host="localhost"
)

cur = conn.cursor()

# SELECT para verificar
query_select = "SELECT * FROM Vila;"
cur.execute(query_select)
df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
print(df)
# df.to_csv("vila.csv", index=False)
# print("CSV gerado com sucesso!")

cur.close()
conn.close()

  id_vila           nome
0     001  Vila da Folha
1     002  Vila da Areia
2     003  Vila da Névoa


In [ ]:
# conn = psycopg2.connect(
#     dbname=bd_name,
#     user="postgres",
#     password="postgres",
#     host="localhost"
# )

# cur = conn.cursor()

# # 1. INSERT
# query_insert = """INSERT INTO Ninja (id_ninja, nome, idade, ranking,tipo, id_vila, id_cla) VALUES
# ('001', 'Itachi Uchiha', 18, 'S', 'J', '001', '001'),
# ('002', 'Mei Terumi', 31, 'S','J' , '003', '002'),
# ('003', 'Gaara', 18, 'S','J' , '002', '003');"""
# cur.execute(query_insert)
# conn.commit()  # ← Salva os dados!
# print("ninjas inseridas com sucesso!")

# # 2. SELECT para verificar (agora sim, fetchall())
# query_select = "SELECT * FROM Ninja;"
# cur.execute(query_select)
# df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
# print(df)
# df.to_csv("ninja.csv", index=False)
# print("CSV gerado com sucesso!")

# cur.close()
# conn.close()

In [ ]:
conn = psycopg2.connect(
    dbname=bd_name,
    user="postgres",
    password="postgres",
    host="localhost"
)

cur = conn.cursor()

query = """
SELECT
    v.nome AS vila,
    c.nome AS clan,
    n.nome AS ninja,
    n.idade,
    n.ranking,
    n.tipo
FROM Ninja n
JOIN Vila v ON n.id_vila = v.id_vila
JOIN Cla c ON n.id_cla = c.id_cla
ORDER BY v.nome, c.nome, n.nome;

"""

cur.execute(query)
df = pd.DataFrame(cur.fetchall(), columns=[desc[0] for desc in cur.description])
print(df)
# df.to_csv("ninjas_completos.csv", index=False)
# print("CSV gerado com sucesso!")

             vila      clan                ninja  idade ranking tipo
0   Vila da Areia   Aburame          Suna Médico     23       B    G
1   Vila da Areia   Aburame       Suna Shinobi B     21       C    C
2   Vila da Areia  Akimichi              Kankuro     17       A    J
3   Vila da Areia  Akimichi                Shira     20       A    C
4   Vila da Areia   Inuzuka       Suna Shinobi A     19       C    G
5   Vila da Areia      Nara                 Baki     35       A    J
6   Vila da Areia      Nara               Temari     17       A    J
7   Vila da Areia     Senju                Gaara     16       S    J
8   Vila da Areia  Yamanaka                 Maki     18       B    G
9   Vila da Areia  Yamanaka            Yashamaru     30       B    C
10  Vila da Folha   Aburame        Shibi Aburame     35       A    J
11  Vila da Folha   Aburame        Shino Aburame     16       A    C
12  Vila da Folha  Akimichi       Choji Akimichi     16       C    G
13  Vila da Folha  Akimichi       

In [ ]:
# as consultas adicionadas até agora foram para testar! precisa:
# 1. adicionar alguns dados nas tabelas que vão ser usadas para o dw
# 2. fazer aquele esquema do dw